In [ ]:
# !pip install feedparser 
# !pip install newspaper3k 
# !pip install konlpy 
# !pip install bs4 
# !sudo apt-get install -y fonts-nanum 
# !sudo fc-cache -fv 
# !rm ~/.cache/matplotlib -rf 
import feedparser 
from newspaper import Article 
from konlpy.tag import Okt 
from collections import Counter
from operator import eq 
import numpy as np 
from bs4 import BeautifulSoup

urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml",
        "http://rss.etnews.com/02.xml"]  # http://rss.etnews.com/02.xml -> 경제 금융에 대한 rss

def crawl_rss(urls):
  arr_rss = []
  for url in urls:
    print("[Crawl RSS]", url)
    parse_rss = feedparser.parse(url)
    for p in parse_rss.entries:
      arr_rss.append({'title':p.title, 'link':p.link})
  return arr_rss

def crawl_article(url, language='ko'):
  print("[Crawl Article]", url)
  var_article = Article(url, language=language)
  var_article.download()
  var_article.parse()
  return var_article.title, var_article.text


# Starting Point
article_list = crawl_rss(urls)
print(article_list)

for article in article_list:
  _, text = crawl_article(article['link'])
  article['text'] = text

In [8]:
# text를 분석한다!
def get_keywords(text, nKeywords=50): # 키워드 숫자 50개 밑에 get_keywords(article['text], 숫자) 로 키워드수 조절 가능 nKeywords=50 -> default 50개로 설정
  num_unique_words = 0
  num_most_freq = 0
  ranking = 0
  spliter = Okt()
  nouns = spliter.nouns(text) # article['text] --> text 인 list로 받기 때문에 nouns도 list로 저장 (명사만 추출) 
  count = Counter(nouns) # 추출한 명사 개수 세기
  return_list = []
  for n, c in count.most_common(nKeywords):
    ranking = ranking + 1
    temp = {'keyword': n, 'count': c, 'ranking': ranking}
    return_list.append(temp)
    num_unique_words = num_unique_words + 1
    if num_unique_words == 1:
      num_most_freq = c
  return num_unique_words, num_most_freq, return_list

def Howmanywords(request, keyword):
  nWords = 0
  nRanking = 0
  for n in keyword:
    noun = n['keyword']
    count = n['count']
    rank = n['ranking']
    if eq(noun, request): # request -> 쿼리
      nWords = count # 쿼리로 입력받은 것에 의한 검색 keyword
      nRanking = rank
  return nWords, nRanking


print('[Parsing Text]')
for article in article_list:
  num_unique_words, num_most_freq, keywords = get_keywords(article['text'])
  article['keywords'] = keywords
  article['num_unique_words'] = num_unique_words
  article['num_most_freq'] = num_most_freq


[Parsing Text]


In [16]:
print('Enter your query:')
query = input()
for article in article_list:
  n, _ = Howmanywords(query, article['keywords'])
  if n!=0:
    print('[TF]:', n, '[Title]:', article['title'], '[link]:', article['link'], '\n')

Enter your query:
바이오
[TF]: 3 [Title]: 조정우 SK바이오팜 사장 "디지털치료제 시장 진출" [link]: https://www.etnews.com/20220324000225 

[TF]: 7 [Title]: SK바이오팜 '세노바메이트', 40배 성장 '기염' [link]: https://www.etnews.com/20220324000197 

[TF]: 3 [Title]: 조정우 SK바이오팜 사장 "디지털치료제 시장 진출" [link]: https://www.etnews.com/20220324000225 

[TF]: 1 [Title]: JW중외제약, 온코크로스와 AI 활용 혁신신약 공동개발 [link]: https://www.etnews.com/20220324000207 

[TF]: 3 [Title]: 조정우 SK바이오팜 사장 "디지털치료제 시장 진출" [link]: https://www.etnews.com/20220324000225 

[TF]: 7 [Title]: SK바이오팜 '세노바메이트', 40배 성장 '기염' [link]: https://www.etnews.com/20220324000197 

[TF]: 3 [Title]: [ET라씨로] 에스엘바이오닉스, 41억원 유상증자 결정...주가 15%대↑ [link]: https://www.etnews.com/20220324000065 

